<a href="https://colab.research.google.com/github/keivernunez/ARSET_ML_Fundamentals/blob/main/Clase5/Note3_de_4_Spacy_Transfrormer_b%C3%A1sico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos Transformer para Traduccion Automatica

En este cuaderno se explora la implementacion de un modelo Transformer para la traduccion automatica de portugues a ingles usando SpaCy con integracion de transformers de Hugging Face. Se utiliza un conjunto de datos paralelo y se aprovecha la eficiencia de SpaCy para inferencia rapida.

## Objetivos de Aprendizaje
- Comprender la arquitectura Transformer y sus ventajas sobre modelos recurrentes.
- Usar SpaCy con transformers preentrenados para tareas de traduccion.
- Preprocesar datos de texto para modelado secuencia a secuencia.
- Realizar inferencia para generar traducciones con mayor velocidad.

**Nota:** Este cuaderno esta disenado con fines didacticos. Cada seccion incluye explicaciones detalladas, fragmentos de codigo comentados y visualizaciones para facilitar la comprension. Basado en el trabajo seminal "Attention is All You Need" (Vaswani et al., 2017), se incorporan practicas recomendadas para el procesamiento de lenguaje natural. Se refactoriza para usar SpaCy, que es mas rapido para inferencia.

**Conceptos Clave a Recordar:**
- **Transformer:** Arquitectura basada en atencion que procesa secuencias en paralelo, superando limitaciones de modelos RNN/LSTM.
- **Atencion Multi-Cabeza:** Mecanismo que permite capturar multiples relaciones semanticas simultaneamente.
- **SpaCy con Transformers:** Proporciona inferencia rapida y eficiente para modelos preentrenados.

**Pregunta para Reflexion:** ¿Por que la paralelizacion en Transformers mejora la eficiencia computacional en comparacion con modelos recurrentes?

## 1. Importacion de Bibliotecas

**Explicacion:** Se importan bibliotecas esenciales para el manejo de datos y modelado con SpaCy. SpaCy facilita la integracion de transformers para NLP eficiente.

In [ ]:
# Se instalan las librerías actualizadas - Reiniciar el runtime si es necesario

%pip install -U pip wheel setuptools -q
%pip install spacy spacy-transformers transformers sacremoses -q

In [ ]:
import spacy
import spacy_transformers
from transformers import pipeline
import tensorflow_datasets as tfds
from IPython.display import Markdown, display
import torch
mixed_precision.set_global_policy("mixed_float16")
print("✅ Bibliotecas importadas con exito.")

In [ ]:
# 🚫 Suprimir todas las advertencias y registros
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" # Ocultar información/advertencias/errores de TensorFlow
os.environ["TRANSFORMERS_VERBOSITY"] = "error" # Ocultar registros de Transformers
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Evitar advertencia de paralelismo del tokenizador
# Opcional: reducir ruido de stderr en Python
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("huggingface_hub").setLevel(logging.ERROR)
print("✅ Todas las advertencias y registros suprimidos.")

✅ Todas las advertencias y registros suprimidos.


## Configuracion de GPU

**Explicacion:** Configuramos el entorno para utilizar GPU si esta disponible, mejorando los tiempos de inferencia.

In [ ]:
device = 0 if torch.cuda.is_available() else -1
print(f'Using device: {"GPU" if device == 0 else "CPU"}')

Using device: CPU


## 2. Carga del Conjunto de Datos

**Explicacion:** Se carga el conjunto TED Talks para traduccion portugues-ingles. Este corpus paralelo es ideal para tareas seq2seq.

**Pregunta para Reflexion:** ¿Que desafios presenta un corpus multilingue en terminos de preprocesamiento?

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
from transformers import pipeline
import spacy

device = 0  # GPU si existe, -1 para CPU

# Cargar dataset de ejemplo portugués → inglés
examples, metadata = tfds.load(
    "ted_hrlr_translate/pt_to_en",
    with_info=True,
    as_supervised=True
)
train_examples, val_examples = examples["train"], examples["validation"]

## 3. Configuracion del Modelo con SpaCy

**Explicacion:** Usamos SpaCy con un transformer preentrenado para traduccion. Cargamos un modelo de Hugging Face para PT-EN.

In [ ]:
# Cargar modelo de traduccion con Hugging Face via transformers (integrable con SpaCy)
translator = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-mul-en",
    device=device
)

# Integración opcional con SpaCy
nlp = spacy.blank("pt")

if not spacy.tokens.Doc.has_extension("translation"):
    spacy.tokens.Doc.set_extension("translation", default=None)

@nlp.component("translator")
def translate_component(doc):
    translation = translator(doc.text)[0]["translation_text"]
    doc._.translation = translation
    return doc

nlp.add_pipe("translator")

# Ejemplo de uso con el dataset TFDS
for pt_text, en_text in train_examples.take(3):
    pt_text = pt_text.numpy().decode("utf-8")
    en_text = en_text.numpy().decode("utf-8")
    doc = nlp(pt_text)
    print(f"\n🔹 Portugués: {pt_text}")
    print(f"🔸 Traducción predicha: {doc._.translation}")
    print(f"✅ Traducción real: {en_text}")



🔹 Portugués: e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
🔸 Traducción predicha: and when we improve the search , we take the only advantage of printing , which is serendipability .
✅ Traducción real: and when you improve searchability , you actually take away the one advantage of print , which is serendipity .

🔹 Portugués: mas e se estes fatores fossem ativos ?
🔸 Traducción predicha: But what if these factors were active?
✅ Traducción real: but what if it were active ?

🔹 Portugués: mas eles não tinham a curiosidade de me testar .
🔸 Traducción predicha: but they didn't have the curiosity to test me.
✅ Traducción real: but they did n't test for curiosity .


## 4. Preparacion de Datos e Inferencia

**Explicacion:** Procesamos ejemplos para inferencia rapida con SpaCy.

In [ ]:
# Ejemplo de inferencia
for pt, en in val_examples.take(5):
    doc = nlp(pt.numpy().decode('utf-8'))
    print(f'Original (PT): {pt.numpy().decode("utf-8")}')
    print(f'Traduccion: {doc._.translation}')
    print(f'Referencia (EN): {en.numpy().decode("utf-8")}\n')

## 5. Evaluacion

**Explicacion:** Evaluar la precision de traduccion usando metricas como BLEU.

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Evaluar en validacion
bleu_scores = []
for pt, en in val_examples.take(100):
    doc = nlp(pt.numpy().decode('utf-8'))
    pred = doc._.translation.split()
    ref = [en.numpy().decode('utf-8').split()]
    bleu_scores.append(sentence_bleu(ref, pred))

print(f'BLEU promedio: {np.mean(bleu_scores)}')

BLEU promedio: 0.20024389941688392
